<a href="https://colab.research.google.com/github/GrzegorzMeller/AlgorithmsForMassiveData/blob/master/FOREST_COVER_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercises 30/04
In the 30/04 lab lecture we will focus on data scaling.

Data scaling is a common preprocessing that is performed on datasets where data is represented with different scales.

Several scaling methods are available, where the most common are:

Normalization. It is basically a rescaling of the data so that the values are within the range  [0,1] .
Standardization. It consists of rescaling the distribution of the observed values to zero mean and unit standard deviation. In the literature, this process is sometimes referred to as whitening.
In the lab lecture we will see how to perform data scaling and why data scaling is important for neural networks.

Meanwhile, address the following exercises.

Build three feed-forward neural network models with one or more layers as follows:

first network: use the original raw data,
second network: use normalized data,
third network: use standardized data.
Do not use convolutional layers. Exploit the methods we saw in the last lab lecture to properly train the networks (for instance, techniques to avoid overfitting). use TensorBoard to assess the performance.

Finally, address the following questions:

Which network reaches the best performance?
Do you notice the difference in performance when scaling the data?
Which scaling method is the best? Can you guess why?

Download the Forest Cover Type Prediction dataset.

The dataset contains tree observations from four areas of the Roosevelt National Forest in Colorado. All observations are cartographic variables (no remote sensing) from 30 meter x 30 meter sections of forest. The task is to predict the forest cover type (the predominant kind of tree cover) from strictly cartographic variables (as opposed to remotely sensed data). More info related to the dataset are available here.

Download the dataset in the Google Colab environment using curl as follows:

In [1]:
!curl http://bodini.di.unimi.it/teaching/ADM_files/covtype.csv --output covtype.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 71.6M  100 71.6M    0     0  13.6M      0  0:00:05  0:00:05 --:--:-- 19.1M


In [2]:
import pandas as pd

df = pd.read_csv('covtype.csv')
x = df[df.columns[:-1]]
y = df.Cover_Type

print(x, y)

        Elevation  Aspect  Slope  ...  Soil_Type38  Soil_Type39  Soil_Type40
0            2596      51      3  ...            0            0            0
1            2590      56      2  ...            0            0            0
2            2804     139      9  ...            0            0            0
3            2785     155     18  ...            0            0            0
4            2595      45      2  ...            0            0            0
...           ...     ...    ...  ...          ...          ...          ...
581007       2396     153     20  ...            0            0            0
581008       2391     152     19  ...            0            0            0
581009       2386     159     17  ...            0            0            0
581010       2384     170     15  ...            0            0            0
581011       2383     165     13  ...            0            0            0

[581012 rows x 54 columns] 0         5
1         5
2         2
3         2


In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state = 90)
print(x_train.shape,y_train.shape, x_test.shape, y_test.shape)

(406708, 54) (406708,) (174304, 54) (174304,)


In [9]:
import tensorflow as tf
from tensorflow import keras
#neural network implementation
model = keras.Sequential([
    keras.layers.Dense(64, activation="relu", input_shape=(x_train.shape[1],)), 

    keras.layers.Dense(64, activation="relu"),


    keras.layers.Dense(8, activation="softmax")]) 
  

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                3520      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 520       
Total params: 8,200
Trainable params: 8,200
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(x_train,
                    y_train,
                    batch_size=60,
                    epochs=30,
                    validation_data=(x_test,y_test),
                    )

Epoch 1/30
6779/6779 [==============================] - 12s 2ms/step - loss: 2.8001 - accuracy: 0.5813 - val_loss: 0.9136 - val_accuracy: 0.7024
Epoch 2/30
6779/6779 [==============================] - 12s 2ms/step - loss: 1.0783 - accuracy: 0.6465 - val_loss: 0.9105 - val_accuracy: 0.6910
Epoch 3/30
6779/6779 [==============================] - 12s 2ms/step - loss: 0.7799 - accuracy: 0.6835 - val_loss: 0.6595 - val_accuracy: 0.7248
Epoch 4/30
6779/6779 [==============================] - 12s 2ms/step - loss: 0.6811 - accuracy: 0.7080 - val_loss: 0.6581 - val_accuracy: 0.7118
Epoch 5/30
6779/6779 [==============================] - 12s 2ms/step - loss: 0.6378 - accuracy: 0.7237 - val_loss: 0.6260 - val_accuracy: 0.7205
Epoch 6/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.6084 - accuracy: 0.7364 - val_loss: 0.5763 - val_accuracy: 0.7538
Epoch 7/30
6779/6779 [==============================] - 12s 2ms/step - loss: 0.5870 - accuracy: 0.7461 - val_loss: 0.6116 - val_ac

In [23]:
#data normalization
import sklearn
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
scaler.fit(x_train)
x_train_norm = scaler.transform(x_train)

scaler2 = preprocessing.MinMaxScaler()
scaler2.fit(x_test)
x_test_norm = scaler2.transform(x_test)

print(x_train_norm, x_test_norm)
print(x_train_norm.shape, x_test_norm.shape)

[[0.5817909  0.36111111 0.16666667 ... 0.         0.         0.        ]
 [0.82791396 0.28611111 0.24242424 ... 0.         0.         1.        ]
 [0.3991996  0.51666667 0.25757576 ... 0.         0.         0.        ]
 ...
 [0.54677339 0.76388889 0.1969697  ... 0.         0.         0.        ]
 [0.58929465 0.06944444 0.10606061 ... 0.         0.         0.        ]
 [0.53176588 0.05       0.16666667 ... 0.         0.         0.        ]] [[0.65912519 0.025      0.11290323 ... 0.         0.         0.        ]
 [0.42182001 0.11944444 0.14516129 ... 0.         0.         0.        ]
 [0.51080945 0.09166667 0.19354839 ... 0.         0.         0.        ]
 ...
 [0.62242333 0.95       0.30645161 ... 0.         0.         0.        ]
 [0.39366516 0.00277778 0.14516129 ... 0.         0.         0.        ]
 [0.64856712 0.30555556 0.14516129 ... 0.         0.         0.        ]]
(406708, 54) (174304, 54)


In [24]:
history2 = model.fit(x_train_norm,
                    y_train,
                    batch_size=60,
                    epochs=30,
                    validation_data=(x_test_norm,y_test),
                    )

Epoch 1/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.6034 - accuracy: 0.7472 - val_loss: 0.5396 - val_accuracy: 0.7670
Epoch 2/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.5219 - accuracy: 0.7760 - val_loss: 0.5074 - val_accuracy: 0.7784
Epoch 3/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.4975 - accuracy: 0.7855 - val_loss: 0.4874 - val_accuracy: 0.7907
Epoch 4/30
6779/6779 [==============================] - 14s 2ms/step - loss: 0.4821 - accuracy: 0.7930 - val_loss: 0.4846 - val_accuracy: 0.7895
Epoch 5/30
6779/6779 [==============================] - 16s 2ms/step - loss: 0.4705 - accuracy: 0.7990 - val_loss: 0.4626 - val_accuracy: 0.8029
Epoch 6/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.4604 - accuracy: 0.8032 - val_loss: 0.4562 - val_accuracy: 0.8048
Epoch 7/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.4535 - accuracy: 0.8064 - val_loss: 0.4609 - val_ac

In [21]:
#data standarization
scaler3 = preprocessing.StandardScaler()
scaler3.fit(x_train)
x_train_scaled = scaler3.transform(x_train)

scaler4 = preprocessing.StandardScaler()
scaler4.fit(x_test)
x_test_scaled = scaler4.transform(x_test)

print(x_train_scaled, x_test_scaled)
print(x_train_scaled.shape, x_test_scaled.shape)

[[ 0.22236642 -0.22863876 -0.41250293 ... -0.16593195 -0.15563993
  -0.12424668]
 [ 1.98049037 -0.46998891  0.25545256 ... -0.16593195 -0.15563993
   8.0485045 ]
 [-1.08193285  0.2719393   0.38904365 ... -0.16593195 -0.15563993
  -0.12424668]
 ...
 [-0.02777316  1.06750088 -0.14532074 ... -0.16593195 -0.15563993
  -0.12424668]
 [ 0.27596776 -1.16722264 -0.94686732 ... -0.16593195 -0.15563993
  -0.12424668]
 [-0.13497584 -1.2297949  -0.41250293 ... -0.16593195 -0.15563993
  -0.12424668]] [[ 0.78692786 -1.31091915 -0.95281686 ... -0.16601249 -0.15688363
  -0.12225904]
 [-0.89692023 -1.00738455 -0.68595149 ... -0.16601249 -0.15688363
  -0.12225904]
 [-0.26547719 -1.09665943 -0.28565345 ... -0.16601249 -0.15688363
  -0.12225904]
 ...
 [ 0.52650221  1.66193439  0.64837533 ... -0.16601249 -0.15688363
  -0.12225904]
 [-1.09669882 -1.38233905 -0.68595149 ... -0.16601249 -0.15688363
  -0.12225904]
 [ 0.71201089 -0.40924285 -0.68595149 ... -0.16601249 -0.15688363
  -0.12225904]]
(406708, 54) (17

In [22]:
history3 = model.fit(x_train_scaled,
                    y_train,
                    batch_size=60,
                    epochs=30,
                    validation_data=(x_test_scaled,y_test),
                    )

Epoch 1/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.8226 - accuracy: 0.7251 - val_loss: 0.6203 - val_accuracy: 0.7438
Epoch 2/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.5885 - accuracy: 0.7540 - val_loss: 0.5556 - val_accuracy: 0.7651
Epoch 3/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.5474 - accuracy: 0.7659 - val_loss: 0.5369 - val_accuracy: 0.7725
Epoch 4/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.5289 - accuracy: 0.7730 - val_loss: 0.5250 - val_accuracy: 0.7748
Epoch 5/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.5162 - accuracy: 0.7784 - val_loss: 0.5148 - val_accuracy: 0.7776
Epoch 6/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.5055 - accuracy: 0.7828 - val_loss: 0.5019 - val_accuracy: 0.7857
Epoch 7/30
6779/6779 [==============================] - 13s 2ms/step - loss: 0.4975 - accuracy: 0.7879 - val_loss: 0.4952 - val_ac